<a href="https://colab.research.google.com/github/liupengzhouyi/LearningPytorch/blob/master/20200310/ConvolutionNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution Network

## import package

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## setting device

In [0]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## setting super parameters

In [0]:
# Hyper parameters
num_epochs = 3
num_classes = 10
batch_size = 100
learning_rate = 0.001

## download data

In [0]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

## create data set

In [0]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

## Create Network

- 1  * 28 * 28
- 16 * 14 * 14
- 32 * 7  * 7

In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

## Instantiation Model

In [0]:
model = ConvNet(num_classes).to(device)

## Create Loss Function

In [0]:
criterion = nn.CrossEntropyLoss()

## Create optimizer

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train The model

In [10]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/3], Step [100/600], Loss: 0.1190
Epoch [1/3], Step [200/600], Loss: 0.1454
Epoch [1/3], Step [300/600], Loss: 0.1272
Epoch [1/3], Step [400/600], Loss: 0.0697
Epoch [1/3], Step [500/600], Loss: 0.0295
Epoch [1/3], Step [600/600], Loss: 0.0211
Epoch [2/3], Step [100/600], Loss: 0.0328
Epoch [2/3], Step [200/600], Loss: 0.0654
Epoch [2/3], Step [300/600], Loss: 0.0407
Epoch [2/3], Step [400/600], Loss: 0.0572
Epoch [2/3], Step [500/600], Loss: 0.0106
Epoch [2/3], Step [600/600], Loss: 0.0224
Epoch [3/3], Step [100/600], Loss: 0.0488
Epoch [3/3], Step [200/600], Loss: 0.0149
Epoch [3/3], Step [300/600], Loss: 0.0355
Epoch [3/3], Step [400/600], Loss: 0.0076
Epoch [3/3], Step [500/600], Loss: 0.0263
Epoch [3/3], Step [600/600], Loss: 0.0154


## Test the model

In [14]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.83 %


## Save the model checkpoint



In [0]:
torch.save(model.state_dict(), 'model.ckpt')

over!